In [1]:
import requests

In [2]:
standings_url = "https://fbref.com/en/comps/9/2023-2024/2023-2024-Premier-League-Stats"

In [3]:
data = requests.get(standings_url)

In [4]:
from bs4 import BeautifulSoup
import time

In [5]:
soup = BeautifulSoup(data.text, 'html.parser')
time.sleep(5)


In [6]:
standing_table = soup.select("table.stats_table")[0]

In [7]:
links = standing_table.find_all('a')

In [8]:
links = [l.get('href') for l in links]

In [9]:
links = [ l for l in links if '/squads/' in l]

In [10]:
teams_url = [f"https://fbref.com{l}" for l in links]

In [11]:
team_url = teams_url[0]
data = requests.get(team_url)

In [12]:
import pandas as pd
from io import StringIO
html_file = StringIO(data.text)
matches = pd.read_html(html_file , match="Scores & Fixtures")
matches[0].head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Opp Formation,Referee,Match Report,Notes
0,2023-08-06,16:00,Community Shield,FA Community Shield,Sun,Neutral,D,1 (1),1 (4),Arsenal,NaN,NaN,55,81145.0,Kyle Walker,4-2-3-1,4-3-3,Stuart Attwell,Match Report,NaN
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,1.9,0.3,65,21572.0,Kevin De Bruyne,4-2-3-1,5-4-1,Craig Pawson,Match Report,NaN
2,2023-08-16,22:00,Super Cup,UEFA Super Cup,Wed,Home,D,1 (5),1 (4),es Sevilla,NaN,NaN,74,NaN,Kyle Walker,4-2-3-1,4-2-3-1,François Letexier,Match Report,NaN
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,1.0,0.3,59,53419.0,Kyle Walker,4-2-3-1,4-3-3,Robert Jones,Match Report,NaN
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,3.5,0.7,79,31336.0,Kyle Walker,4-2-3-1,3-5-2,Jarred Gillett,Match Report,NaN


In [13]:
soup = BeautifulSoup(data.text)
time.sleep(5)

In [14]:
links = soup.find_all('a')

In [15]:
links = [l.get("href") for l in links]

In [16]:
links = [ l for l in links if l and "/all_comps/shooting/" in l]

In [17]:
data = requests.get(f"https://fbref.com{links[0]}")

In [18]:
html_file = StringIO(data.text)
shooting = pd.read_html(html_file , match="Shooting")[0]
shooting.columns = shooting.columns.droplevel()
shooting.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2023-08-06,16:00,Community Shield,FA Community Shield,Sun,Neutral,D,1 (1),1 (4),Arsenal,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Match Report
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,13.9,0.0,0,0,1.9,1.9,0.12,1.1,1.1,Match Report
2,2023-08-16,22:00,Super Cup,UEFA Super Cup,Wed,Home,D,1 (5),1 (4),es Sevilla,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Match Report
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,17.9,0.0,0,0,1.0,1.0,0.07,0.0,0.0,Match Report
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,17.3,2.0,0,1,3.5,2.8,0.10,-1.5,-0.8,Match Report


In [19]:
team_data = matches[0].merge(shooting[["Date" , "Sh" , "SoT","Dist","FK","PK","PKatt"]] , on="Date")

In [20]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Opp Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2023-08-06,16:00,Community Shield,FA Community Shield,Sun,Neutral,D,1 (1),1 (4),Arsenal,...,4-3-3,Stuart Attwell,Match Report,NaN,8,4,NaN,NaN,0,0
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,5-4-1,Craig Pawson,Match Report,NaN,17,8,13.9,0.0,0,0
2,2023-08-16,22:00,Super Cup,UEFA Super Cup,Wed,Home,D,1 (5),1 (4),es Sevilla,...,4-2-3-1,François Letexier,Match Report,NaN,23,7,NaN,NaN,0,0
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,4-3-3,Robert Jones,Match Report,NaN,14,4,17.9,0.0,0,0
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,3-5-2,Jarred Gillett,Match Report,NaN,29,9,17.3,2.0,0,1


In [21]:
# years = list(range(2025 , 2023 , -1))
all_matches = []

In [22]:
standings_url = "https://fbref.com/en/comps/9/2023-2024/2023-2024-Premier-League-Stats"

In [23]:

data = requests.get(standings_url)
soup = BeautifulSoup(data.text, 'html.parser')
# time.sleep(5)
standing_table = soup.select('table.stats_table')[0]

links = standing_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

# previous_season = soup.select("a.prev")[0].get("href")
# standings_url = f"https://fbref.com{previous_season}"

for team_url in teams_url:
    team_name = team_url.split('/')[-1].replace("-stats", "").replace("-", " ")
    data = requests.get(team_url)
    html_file = StringIO(data.text)
    matches = pd.read_html(html_file, match="Scores & Fixtures")[0]
    soup = BeautifulSoup(data.text, 'html.parser')
    # time.sleep(5)
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and "all_comps/shooting/" in l]
    if not links:
        continue
    data = requests.get(f"https://fbref.com{links[0]}")
    html_file = StringIO(data.text)
    shooting = pd.read_html(html_file, match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()
    try:
        team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
    except ValueError:
        continue
    team_data = team_data[team_data["Comp"] == "Premier League"]

    # team_data["Season"] = year
    team_data["Team"] = team_name
    all_matches.append(team_data)
    time.sleep(10)
        


In [24]:
len(all_matches)

20

In [25]:
match_df = pd.concat(all_matches)

In [26]:
match_df.columns = [c.lower() for c in match_df.columns]

In [27]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,referee,match report,notes,sh,sot,dist,fk,pk,pkatt,team
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,Craig Pawson,Match Report,NaN,17.0,8.0,13.9,0.0,0,0,Manchester City Stats
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,Robert Jones,Match Report,NaN,14.0,4.0,17.9,0.0,0,0,Manchester City Stats
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,Jarred Gillett,Match Report,NaN,29.0,9.0,17.3,2.0,0,1,Manchester City Stats
5,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Home,W,5,1,Fulham,...,Michael Oliver,Match Report,NaN,6.0,4.0,14.8,0.0,1,1,Manchester City Stats
6,2023-09-16,15:00,Premier League,Matchweek 5,Sat,Away,W,3,1,West Ham,...,Andy Madley,Match Report,NaN,29.0,13.0,16.4,1.0,0,0,Manchester City Stats
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,2024-04-24,20:00,Premier League,Matchweek 29,Wed,Away,L,2,4,Manchester Utd,...,Michael Salisbury,Match Report,NaN,10.0,4.0,17.8,1.0,0,0,Sheffield United Stats
37,2024-04-27,15:00,Premier League,Matchweek 35,Sat,Away,L,1,5,Newcastle Utd,...,Tony Harrington,Match Report,NaN,15.0,4.0,13.5,0.0,0,0,Sheffield United Stats
38,2024-05-04,15:00,Premier League,Matchweek 36,Sat,Home,L,1,3,Nott'ham Forest,...,Chris Kavanagh,Match Report,NaN,16.0,4.0,18.1,0.0,1,1,Sheffield United Stats
39,2024-05-11,15:00,Premier League,Matchweek 37,Sat,Away,L,0,1,Everton,...,Stuart Attwell,Match Report,NaN,13.0,1.0,21.0,0.0,0,0,Sheffield United Stats


In [28]:
match_df.to_csv("matches1.csv")

In [35]:
df = pd.read_csv("matches1.csv")
df.dropna()  # Remove missing values
df.fillna(0)  # Replace missing values with 0

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,referee,match report,notes,sh,sot,dist,fk,pk,pkatt,team
0,1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,...,Craig Pawson,Match Report,0.0,17.0,8.0,13.9,0.0,0,0,Manchester City Stats
1,3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,...,Robert Jones,Match Report,0.0,14.0,4.0,17.9,0.0,0,0,Manchester City Stats
2,4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,...,Jarred Gillett,Match Report,0.0,29.0,9.0,17.3,2.0,0,1,Manchester City Stats
3,5,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Home,W,5,1,...,Michael Oliver,Match Report,0.0,6.0,4.0,14.8,0.0,1,1,Manchester City Stats
4,6,2023-09-16,15:00,Premier League,Matchweek 5,Sat,Away,W,3,1,...,Andy Madley,Match Report,0.0,29.0,13.0,16.4,1.0,0,0,Manchester City Stats
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,36,2024-04-24,20:00,Premier League,Matchweek 29,Wed,Away,L,2,4,...,Michael Salisbury,Match Report,0.0,10.0,4.0,17.8,1.0,0,0,Sheffield United Stats
756,37,2024-04-27,15:00,Premier League,Matchweek 35,Sat,Away,L,1,5,...,Tony Harrington,Match Report,0.0,15.0,4.0,13.5,0.0,0,0,Sheffield United Stats
757,38,2024-05-04,15:00,Premier League,Matchweek 36,Sat,Home,L,1,3,...,Chris Kavanagh,Match Report,0.0,16.0,4.0,18.1,0.0,1,1,Sheffield United Stats
758,39,2024-05-11,15:00,Premier League,Matchweek 37,Sat,Away,L,0,1,...,Stuart Attwell,Match Report,0.0,13.0,1.0,21.0,0.0,0,0,Sheffield United Stats


In [36]:
# df = pd.read_csv('matches.csv')  # Replace with your actual file name
df = df.drop(columns=['notes'])  # Replace 'ColumnName' with the actual column name
df = df.drop(columns=['match report'])
df = df.drop(columns=['pkatt'])
df = df.drop(columns=['pk'])
df = df.drop(columns=['fk'])
df.to_csv('new_matches1.csv', index=False)  # Save without the index column
df

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,poss,attendance,captain,formation,opp formation,referee,sh,sot,dist,team
0,1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,...,65.0,21572.0,Kevin De Bruyne,4-2-3-1,5-4-1,Craig Pawson,17.0,8.0,13.9,Manchester City Stats
1,3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,...,59.0,53419.0,Kyle Walker,4-2-3-1,4-3-3,Robert Jones,14.0,4.0,17.9,Manchester City Stats
2,4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,...,79.0,31336.0,Kyle Walker,4-2-3-1,3-5-2,Jarred Gillett,29.0,9.0,17.3,Manchester City Stats
3,5,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Home,W,5,1,...,68.0,52899.0,Kyle Walker,4-2-3-1,4-3-3,Michael Oliver,6.0,4.0,14.8,Manchester City Stats
4,6,2023-09-16,15:00,Premier League,Matchweek 5,Sat,Away,W,3,1,...,68.0,62475.0,Kyle Walker,4-2-3-1,4-2-3-1,Andy Madley,29.0,13.0,16.4,Manchester City Stats
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,36,2024-04-24,20:00,Premier League,Matchweek 29,Wed,Away,L,2,4,...,29.0,73549.0,Oliver Arblaster,3-5-2,4-2-3-1,Michael Salisbury,10.0,4.0,17.8,Sheffield United Stats
756,37,2024-04-27,15:00,Premier League,Matchweek 35,Sat,Away,L,1,5,...,40.0,52196.0,Oliver Arblaster,3-5-2,4-3-3,Tony Harrington,15.0,4.0,13.5,Sheffield United Stats
757,38,2024-05-04,15:00,Premier League,Matchweek 36,Sat,Home,L,1,3,...,54.0,28516.0,Jack Robinson,3-5-2,4-2-3-1,Chris Kavanagh,16.0,4.0,18.1,Sheffield United Stats
758,39,2024-05-11,15:00,Premier League,Matchweek 37,Sat,Away,L,0,1,...,42.0,39221.0,Jack Robinson,3-5-2,4-4-1-1,Stuart Attwell,13.0,1.0,21.0,Sheffield United Stats
